## simple notebook to get ERA5 data for Kelmarsh wind farm

We often need to fill in gaps for missing on site records. Having datasets like MERRA2 and ERA5 gives us a data source that can be used to build models to fill gaps. This is a simplified example. A professional analyst would convert this notebook to a function that accepted the lat/long for a site, or would develop a loop to get multiple sites and would set the date range for data programmatically.  

This notebook gets ERA5 data for the Kelmarsh wind farm in the UK at lat 52.401461, long -0.943105
using information available from <br>
CDS https://cds.climate.copernicus.eu/how-to-api


This code will not work until you sign up for a cds account and follow the instructions on the CDS page above to get your own api key.

Did you get your own api key yet?

### Key libraries

These are the key libraries and their revision level when this code was authored.  Consider including info about lirbary versions as comments in the script this is the first step in figuring out why the code you ran last month doesn't work this month. e discussions in entry level section of this repo for more details

* python version: 3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0]
* numpy version: 1.26.2
* polars version: 1.17.1
* cdsapi version: 0.7.5
* xarray version: 2023.6.0


In [ ]:
print('python version:', __import__('sys').version)

import math # to use math functions like radians, atan2
from calendar import monthrange # to get the number of days in a month
import time # for time.sleep() to wait for the API to respond
from pathlib import Path # to work with file paths
cwd = Path.cwd()

import numpy as np
print('numpy version:', np.__version__)
import polars as pl # to use polars dataframes for our data
print('polars version:', pl.__version__)
import polars.selectors as cs # to use column selectors in polars
# Set the display width
pl.Config.set_tbl_cols(100)  # Set the number of polars df columns to display when printing
pl.Config.set_tbl_width_chars(200)  # Set the width of polars df columns in characters when printing
import cdsapi # to get the ERA5 data
import pkg_resources
cdsapi_version = pkg_resources.get_distribution("cdsapi").version
print('cdsapi version:', cdsapi_version)
import xarray as xr # to read the netCDF files
print('xarray version:', xr.__version__)



### Identify surrounding 4 grid points

ERA5 data is set up in a 0.25 deg grid for the entire world, we need to find either the closest point, or the surrounding points.  

Since its just as easy to get surrounding points we do this, so we can understand the full context or what's going on around our point of interest (Kelmarsh wind farm).

In [ ]:

def get_surrounding_grid_points(lat, lon, interval=0.25):
    # Calculate the nearest grid point
    nearest_lat = np.round(lat / interval) * interval
    nearest_lon = np.round(lon / interval) * interval

    # Calculate surrounding grid points
    lat_points = [nearest_lat - interval, nearest_lat, nearest_lat + interval]
    lon_points = [nearest_lon - interval, nearest_lon, nearest_lon + interval]

    # Generate all combinations of surrounding grid points
    surrounding_points = [(lat, lon) for lat in lat_points for lon in lon_points]
    
    return surrounding_points

def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Compute differences
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distance in kilometers
    distance = R * c
    return distance

def get_closest_grid_points(lat, lon, num_points=4):
    surrounding_points = get_surrounding_grid_points(lat, lon)
    distances = [(point, haversine(lat, lon, point[0], point[1])) for point in surrounding_points]
    distances.sort(key=lambda x: x[1])
    closest_points = [point for point, distance in distances[:num_points]]
    return closest_points

# Kelmarsh coordinates
lat = 52.40
lon = -0.943
# find the ERA5 grid points closest to Kelmarsh - ERA5 data is on a 0.25 degree grid, both in latitude and longitude
closest_points = get_closest_grid_points(lat, lon)


# Print the closest points and their distances
for point in closest_points:
    distance = np.round(haversine(lat, lon, point[0], point[1]), 3)
    print(f'For point {point}, distance is {distance} km from Kelmarsh at {lat}, {lon}')

### get the data from CDS

As noted above, you have to get an API key first from <br>
CDS https://cds.climate.copernicus.eu/how-to-api




In [ ]:

def download_era5_data(year, month, closest_points, output_dir, max_retries=3):
    # Initialize the CDS API client
    c = cdsapi.Client()

    # Get the number of days in the month
    num_days = monthrange(year, month)[1]

    # Ensure the output directory exists, create if not
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Calculate the expected range from the closest points
    lat_min = min(point[0] for point in closest_points)
    lat_max = max(point[0] for point in closest_points)
    lon_min = min(point[1] for point in closest_points)
    lon_max = max(point[1] for point in closest_points)

    # Request ERA5 data with retry mechanism
    for attempt in range(max_retries):
        try:
            c.retrieve(
                'reanalysis-era5-single-levels',
                {
                    'product_type': 'reanalysis',
                    'format': 'netcdf',  # Options: 'grib' or 'netcdf'
                    'variable': [
                        '2m_temperature', '10m_u_component_of_wind', '10m_v_component_of_wind',
                        'surface_pressure', '100m_u_component_of_wind', '100m_v_component_of_wind'
                    ],
                    'year': str(year),
                    'month': f'{month:02d}',
                    'day': [f'{day:02d}' for day in range(1, num_days + 1)],
                    'time': [
                        '00:00', '01:00', '02:00', '03:00', '04:00', '05:00',
                        '06:00', '07:00', '08:00', '09:00', '10:00', '11:00',
                        '12:00', '13:00', '14:00', '15:00', '16:00', '17:00',
                        '18:00', '19:00', '20:00', '21:00', '22:00', '23:00',
                    ],
                    'area': [
                        lat_max, lon_min, lat_min, lon_max,
                    ],  # North, West, South, East
                },
                output_dir / f'era5_single_levels_{year}{month:02d}.nc'  # Output file name
            )
            print(f'Successfully downloaded data for {year}-{month:02d} into {output_dir}')
            break  # Exit the loop if the download is successful
        except Exception as e:
            print(f'Failed to download data for {year}-{month:02d} on attempt {attempt + 1}: {e}')
            if attempt < max_retries - 1:
                print('Retrying...')
                time.sleep(5)  # Wait for 5 seconds before retrying
            else:
                print('Max retries reached. Moving to the next month.')

# Example usage: Loop through months of a year and download data for each month to our directory
output_dir = cwd / 'era5_netcdf_files'
year = 2020
for month in range(1, 13):
    download_era5_data(year, month, closest_points, output_dir)
    # CDS data is in high demand and it's a busy site. Code can take 10x longer to run during European work hours.
    # From the US, in the evening, this code may take 3-5 minutes to run.
    # from the US, in the morning, this code may take 30-50 minutes to run.

### combine nc files into polars df

The data is in netcdf format, so we mine the nc files and make a polars df

Why polars?  It's faster, and it more closely follows rules for data structures being in a table format. The syntax is also easier to comprehend.

And it's faster. 


In [4]:

def load_nc_files_to_era5_data(nc_files):
    
    dfs = []
    for nc_file in nc_files:
        # Load the NetCDF file using xarray
        ds = xr.open_dataset(nc_file)
        # Convert xarray.Dataset to a Pandas DataFrame
        df = ds.to_dataframe().reset_index()
        # Convert Pandas DataFrame to Polars DataFrame
        pl_df = pl.from_pandas(df)
        # Append to the list of DataFrames
        dfs.append(pl_df)
    # Concatenate all Polars DataFrames
    combined_df = pl.concat(dfs)
    return combined_df

# make a list of all .nc files downloaded in previous step
# uses same output_dir as previous step
nc_files = list(output_dir.glob('*.nc'))

# Load the NetCDF files into a Polars DataFrame, then clean up 
era5_data = (load_nc_files_to_era5_data(nc_files) # returns a polars dataframe
             .drop(['expver', 'number']) # drops the columns 'expver' and 'number' which didn't have data
             # Date, TimeStamp, latitude, longitude not in table of accepted abbreviations in section 4 of IEC 61400-25-2, 
             # so used section 7.2.4.2 name for TimeStamp, appending UTC to avoid confusion with local time
             # and table 45 names for latitude and longitude
             .rename({'valid_time':'TimeStamp_UTC', 'latitude':'latitude', 'longitude':'longitude', # lat/long names kept
                      'u10':'HorWdU_Alt10m', 'v10':'HorWdV_Alt10m', # some signals marked with 10m meaning 10 minutes,  
                      'u100':'HorWdU_Alt100m', 'v100':'HorWdV_Alt100m', # so add Alt so 10m is altitude AGL above ground level
                      't2m':'EnvTmp_Alt2m', 'sp':'EnvPres_Alt0m'})
             .with_columns(pl.col('TimeStamp_UTC').dt.cast_time_unit('ms').alias('TimeStamp_UTC')) # put ts in ms, needs to be consistent later for joins
             .sort(['TimeStamp_UTC', 'latitude', 'longitude'])
             # consider variables used and if it is reasonable to store as float32 instead of float64 as it takes up 
             # half the space in RAM and you can deal with larger datasets
             # the command below selects columns that are type float (32 or 64) and casts them to float32
             .cast({cs.float():pl.Float32}) )

# running polars data manipulations in a single statement allows the query optimizer to work more effectively, speeding execution.



In [ ]:
# review data for completeness and reasonableness, and ts range
print(era5_data.describe())

In [ ]:
print(era5_data.head())

In [7]:
# Ensure the output directory exists, create it if not
output_dir = cwd / 'era5_data'
output_dir.mkdir(parents=True, exist_ok=True)
era5_data.write_csv(output_dir / 'era5_data.csv', datetime_format='%Y-%m-%d %H:%M:%S')
era5_data.write_parquet(output_dir / 'era5_data.parquet')